# Classification Examples

&nbsp;

<div style="text-align: left;">
    <img src="../utils/1ampere_logo_®_primary_stacked_rgb.png" width="50%"/>    
</div>

<br>

Ampere AI software stack is the software acceleration layer of Ampere Cloud Native Processors specifically dedicated to accelerating AI workloads running on Ampere Processors. Ampere Optimized AI Frameworks include PyTorch, TensorFlow, and ONNXRuntime. This drop-in library seamlessly supports all AI applications developed in the most popular AI frameworks. It works  right out-of-the-box without API changes or any additional coding. Additionally, the Ampere AI software engineering team provides the publicly accessbile Ampere Model Library (AML) for testing and benchmarking the performance ofAmpere Cloud Native Processors for some of the most common AI inference workloads.

Please visit us at https://amperecomputing.com

&nbsp;

## ImageNet Dataset Overview

<div style="text-align: left;">
    <img align="left" src="https://www.image-net.org/static_files/index_files/logo.jpg" alt="nn" style="width: 200px;"/>
</div>


&nbsp;

These examples are using subset of ImageNet classification validation set from year 2012.
ImageNet is a large-scale classification dataset that has been instrumental in advancing computer vision and deep learning research.

More info can be found here: https://image-net.org/

&nbsp;

In [ ]:
import os
import cv2
import time
import subprocess
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from utils.imagenet import ImageNet
import utils.post_processing as pp
import utils.benchmark as bench_utils

LAT_BATCH_SIZE = 1
THROUGHPUT_BATCH_SIZE = 32

## Latency with ResNet-50 v1.5 in fp32 precision

AIO offers a significant speed-up in standard fp32 inference scenarios. AIO exposes
AIO API to control behavior of the optimizer.
This example shows the performance of ResNet-50 v1.5 model in fp32 precision.
Original ResNet paper can be found here: https://arxiv.org/pdf/1512.03385.pdf

In [ ]:
input_shape = (224, 224)
path_to_fp32_model = "resnet_50_v15/resnet_50_v15_tf_fp32.pb"
path_to_fp16_model = "resnet_50_v15/resnet_50_v15_tf_fp16.pb"

In [ ]:
# first let's load the model

graph = tf.compat.v1.Graph()
with graph.as_default():
    graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v1.gfile.GFile(path_to_fp32_model, 'rb') as fid:
        serialized_graph = fid.read()
        graph_def.ParseFromString(serialized_graph)
        tf.compat.v1.import_graph_def(graph_def, name="")

In [ ]:
# ! AIO_NUM_THREADS should be set prior to launching jupyter notebook !

# creating TF config
config = tf.compat.v1.ConfigProto()
config.allow_soft_placement = True
config.intra_op_parallelism_threads = bench_utils.get_intra_op_parallelism_threads()
config.inter_op_parallelism_threads = 1

In [ ]:
# preparing input and output dictionaries

# creation of output dictionary
output_dict = {"softmax_tensor:0": graph.get_tensor_by_name("softmax_tensor:0")}

# initialization of ImageNet dataset
imagenet = ImageNet(
    batch_size=LAT_BATCH_SIZE,
    color_model="RGB",
    pre_processing="VGG",
    is1001classes=True
)

input_array = imagenet.get_input_array(target_shape=input_shape)

# assignment of input image to input tensor
feed_dict = {graph.get_tensor_by_name("input_tensor:0"): input_array}

# for the purpose of visualizing results let's load the image without pre-processing
img = cv2.imread(str(imagenet.path_to_latest_image))

In [ ]:
# running the model with AIO enabled in fp32 precision

tf.AIO.force_enable()

with tf.compat.v1.Session(config=config, graph=graph) as sess:
    # warm-up run
    _ = sess.run(output_dict, feed_dict)

    # actual run
    start = time.time()
    output_aio = sess.run(output_dict, feed_dict)["softmax_tensor:0"]
    finish = time.time()

latency_ms = (finish - start) * 1000
print("\nResNet-50 v1.5 FP32 latency with AIO: {:.0f} ms\n".format(latency_ms))

In [ ]:
# running the model with AIO disabled in fp32 precision

tf.AIO.force_disable()

with tf.compat.v1.Session(config=config, graph=graph) as sess:
    # warm-up run
    _ = sess.run(output_dict, feed_dict)

    # actual run
    start = time.time()
    output_no_aio = sess.run(output_dict, feed_dict)["softmax_tensor:0"]
    finish = time.time()

latency_ms = (finish - start) * 1000
print("\nResNet-50 v1.5 FP32 latency without AIO: {:.0f} ms\n".format(latency_ms))

In [ ]:
# visualizing output

# show the image
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()
print("ResNet-50 v1.5 FP32 predictions with AIO enabled:\n")
print(f"Top-1 prediction: {pp.get_imagenet_names(imagenet.extract_top1(output_aio[0]) + 1)}")
print(f"Top-5 predictions: {pp.get_imagenet_names(imagenet.extract_top5(output_aio[0]) + 1)}")

print("\nResNet-50 v1.5 FP32 predictions with AIO disabled:\n")
print(f"Top-1 prediction: {pp.get_imagenet_names(imagenet.extract_top1(output_no_aio[0]) + 1)}")
print(f"Top-5 predictions: {pp.get_imagenet_names(imagenet.extract_top5(output_no_aio[0]) + 1)}")

## Throughput (BS=32) with ResNet-50 v1.5

In [ ]:
# let's fill array of shape [32, 224, 224, 3] with our image

input_array_bs32 = np.empty([THROUGHPUT_BATCH_SIZE, *input_shape, 3])  # NHWC order
for i in range(THROUGHPUT_BATCH_SIZE):
    input_array_bs32[i] = input_array

In [ ]:
# let's start with fp32 non-aio case

feed_dict = {graph.get_tensor_by_name("input_tensor:0"): input_array_bs32}

In [ ]:
# running the model with AIO disabled in fp32 precision

tf.AIO.force_disable()

with tf.compat.v1.Session(config=config, graph=graph) as sess:
    # warm-up run
    _ = sess.run(output_dict, feed_dict)

    # actual run
    start = time.time()
    _ = sess.run(output_dict, feed_dict)["softmax_tensor:0"]
    finish = time.time()

throughput_no_aio = THROUGHPUT_BATCH_SIZE / (finish - start)

In [ ]:
# we have to load fp16 variant now

graph = tf.compat.v1.Graph()
with graph.as_default():
    graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v1.gfile.GFile(path_to_fp16_model, 'rb') as fid:
        serialized_graph = fid.read()
        graph_def.ParseFromString(serialized_graph)
        tf.compat.v1.import_graph_def(graph_def, name="")

In [ ]:
# preparing input and output dictionaries

# creation of output dictionary
output_dict = {"softmax_tensor:0": graph.get_tensor_by_name("softmax_tensor:0")}

# assignment of input image to input tensor
feed_dict = {graph.get_tensor_by_name("input_tensor:0"): input_array_bs32}

In [ ]:
# running the model with AIO enabled in fp16 precision

tf.AIO.force_enable()

with tf.compat.v1.Session(config=config, graph=graph) as sess:
    # warm-up run
    _ = sess.run(output_dict, feed_dict)

    # actual run
    start = time.time()
    _ = sess.run(output_dict, feed_dict)["softmax_tensor:0"]
    finish = time.time()

throughput_aio = THROUGHPUT_BATCH_SIZE / (finish - start)
print("\nResNet-50 v1.5 FP32 throughput without AIO: {:.0f} fps".format(throughput_no_aio))
print("\nResNet-50 v1.5 FP16 throughput with AIO: {:.0f} fps".format(throughput_aio))